# Training neural networks

In [1]:
import sys

sys.path.append("../NeuralNetworkClasses")
from extract_from_root import *
from NN_class import *
from dataset_loading import *

import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import Subset
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

import itertools
import timeit
import uproot

## Training networks

### Fully-connected neural networks

In [2]:
########### The neural network parametrization ###########

### Neural Network

class network(nn.Module):

    def __init__(self, inch=1, window_size=11):
        
        super().__init__()
        
        self.window=window_size
        
        self.conv11 = nn.Conv2d(inch, 2, kernel_size=3, stride=1, padding=1)
        #self.bn = nn.BatchNorm2d(2)        
        self.conv21 = nn.Conv2d(2, 1, kernel_size=5, stride=1)
        
        self.fc31 = nn.Linear((window_size-4)**2, (window_size-4)**2)
        self.fc41 = nn.Linear((window_size-4)**2, 3)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        
    def forward(self, X):
        
        out = self.relu(self.conv11(X))
        #out = self.bn(out)
        # out = out+self.adjacent
        out = self.relu(self.conv21(out))
        out = out.reshape((-1,(self.window-4)**2))
        out = self.tanh(self.fc31(out))
        out = self.fc41(out)
        
        return out
    
dict_net = {
    "GLOBAL": {
        "epochs": 3,
        "loss_function": nn.MSELoss(),
        "optimizer": optim.Adam,
        "scheduler": optim.lr_scheduler.ReduceLROnPlateau,
        "amp": False, ### Mixed precision training
        "quantization": False, ### Quantize the model to e.g. Int8
        "profiler": True,
    },
    "DATA_OPTIONS": {
        "batchsize_schedule": [0],
        "batchsize_training": [None],
        "batchsize_validation": None,
        "shuffle_every_epoch": True,
        "num_workers": 0,
        "pin_memory": None,
        "copy_to_device": False,
    },
    "AMP_OPTIONS" : {
        "dtype": torch.float16,
    },
    "QUANTIZATION_OPTIONS" : {
        "ONNX": {
            "weight_type": QuantType.QInt8,
            "per_channel": False,
        },
        "PYTORCH": {
            "dtype": torch.qint8,
        },
    },
    "LOSS_OPTIONS": {},
    "OPTIMIZER_OPTIONS": {
        "lr": 0.001,
        "weight_decay": 0
        },
    "SCHEDULER_OPTIONS": {
        "patience": 5,
        "factor": 0.5
        },
    "MACHINE_OPTIONS": {
        "device": 'mps',
        "cpu_threads": None,
        "dtype": torch.float32,
    },
    "ONNX": {
        "export_params": True,
        "opset_version": 17,
        "do_constant_folding": True,
        "input_names": ["input"],
        "output_names": ["output"],
        "dynamic_axes": {
            "input": {0: "batch_size"},
            "output": {0: "batch_size"},
        },
    },
    "PROFILER": {
        "schedule": torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=1),
        "on_trace_ready": torch.profiler.tensorboard_trace_handler('./profiler_log'),
        "record_shapes": True,
        "with_stack": True,
    },
    "OTHER": {
        "verbose": True,
        "n_samples": np.infty,
        "save_at_epochs": [1],
    },
}

dict_data = {
    "GLOBAL" : {
        "transform_data" : False,
        "copy_to_device" : True,
    },
    "SCALERS" : {
        "X_data_scalers" : [('yeo-johnson', preprocessing.PowerTransformer(method='yeo-johnson', standardize=True))],
        "Y_data_scalers" : [('yeo-johnson', preprocessing.PowerTransformer(method='yeo-johnson', standardize=True))],
    },
    "MACHINE_OPTIONS" : {
        "device" : 'mps',
        "dtype_X" : torch.float32,
        "dtype_Y" : torch.float32
    },
    "OTHER" : {
        "num_workers" : 0
    }
}


### define the neural network
NeuralNet = NN(network())

### data preparation

example_data_X = torch.tensor(np.random.random((1000,1,11,11)))
example_data_Y = torch.tensor(np.random.random((1000,3)))
X_train, X_test, y_train, y_test = train_test_split(example_data_X,example_data_Y,test_size=0.1,shuffle=True)

data = DataLoading([X_train, y_train], [X_test, y_test], settings = dict_data)

### evaluate training and validation loss over epochs                   
NeuralNet.training(data, settings = dict_net)

NeuralNet.save_net(avoid_q=True)
NeuralNet.save_onnx(example_data=torch.tensor(X_train[:1]))
NeuralNet.eval()

---  Data settings  ---
{
    "GLOBAL": {
        "transform_data": false,
        "copy_to_device": true
    },
    "SCALERS": {
        "X_data_scalers": [
            [
                "yeo-johnson",
                "PowerTransformer()"
            ]
        ],
        "Y_data_scalers": [
            [
                "yeo-johnson",
                "PowerTransformer()"
            ]
        ]
    },
    "MACHINE_OPTIONS": {
        "device": "mps",
        "dtype_X": "torch.float32",
        "dtype_Y": "torch.float32",
        "amp": false,
        "flexible_data": false
    },
    "OTHER": {
        "num_workers": 0
    }
} 


================ Data preparation ================

No transformation is performed on training data!


RuntimeError: PyTorch is not linked with support for mps devices

In [2]:
test_input = np.load("/lustre/alice/users/csonnab/PhD/jobs/clusterization/NN/training_data/normalized_qCenter/test/input_data.npy")
test_class = np.load("/lustre/alice/users/csonnab/PhD/jobs/clusterization/NN/training_data/normalized_qCenter/test/output_data_class.npy").flatten()
test_reg_label = np.loadtxt("/lustre/alice/users/csonnab/PhD/jobs/clusterization/NN/training_data/normalized_qCenter/test/output_data_reg.txt", dtype=str)
test_reg = np.load("/lustre/alice/users/csonnab/PhD/jobs/clusterization/NN/training_data/normalized_qCenter/test/output_data_reg.npy")

In [3]:
data_mask = test_class > 0

In [4]:
test_input = test_input[data_mask]
test_class = test_class[data_mask]
test_reg = test_reg[data_mask]

In [5]:
class_n_data = list()
for i in range(5):
    class_n_data.append(list())
    for j, label in enumerate(test_reg_label):
        for k in range(i):
            if str(k) in label:
                class_n_data[i].append(j)

In [6]:
class_n_data

[[],
 [0, 5, 10, 15, 20],
 [0, 1, 5, 6, 10, 11, 15, 16, 20, 21],
 [0, 1, 2, 5, 6, 7, 10, 11, 12, 15, 16, 17, 20, 21, 22],
 [0, 1, 2, 3, 5, 6, 7, 8, 10, 11, 12, 13, 15, 16, 17, 18, 20, 21, 22, 23]]

In [7]:
output_data = [torch.tensor(5*i) for i in test_class]

In [8]:
output_data = [i for i in test_class]
for i, lbl_idx in enumerate(test_class):
    output_data[i] = test_reg[i, np.array(class_n_data[lbl_idx])]

In [9]:
input_data = list(zip(test_input, test_class))

In [10]:
### Neural Network

class network(nn.Module):

    def __init__(self, model_classes):
        super(network, self).__init__()
        self.num_models = len(model_classes)
        self.models = nn.ModuleList(model_classes)

    def forward(self, input_data):
        assert input_data[0].size(0) == input_data[1].size(0), "Input and index array must have the same length."

        # Use advanced indexing to select models based on the index array
        selected_models = self.models[input_data[1]]

        # Stack the selected models
        stacked_models = nn.ModuleList(selected_models)

        # Apply models to input data using parallel forward
        output = nn.parallel.parallel_apply(stacked_models, input_data[1].chunk(input_data[0].size(0), dim=0))

        return torch.cat(output, dim=0)


class Model1(nn.Module):
    def __init__(self):
        
        super().__init__()
        
        # Define fully connected layers
        self.fc1 = nn.Linear(7*7*7, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, 5)

        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax()
        self.relu = nn.ReLU()
        
    def forward(self, X):

        # out = self.quant(X)
        X = X.view(-1,7*7*7)
        out = self.relu(self.fc1(X))
        out = self.relu(self.fc2(out))
        out = self.relu(self.fc3(out))
        out = self.relu(self.fc4(out))
        out = self.relu(self.fc5(out))
        out = self.softmax(self.fc6(out))
        return out

class Model2(nn.Module):

    def __init__(self):
        
        super().__init__()
        
        # Define fully connected layers
        self.fc1 = nn.Linear(7*7*7, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, 10)

        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax()
        self.relu = nn.ReLU()
        
    def forward(self, X):

        # out = self.quant(X)
        X = X.view(-1,7*7*7)
        out = self.relu(self.fc1(X))
        out = self.relu(self.fc2(out))
        out = self.relu(self.fc3(out))
        out = self.relu(self.fc4(out))
        out = self.relu(self.fc5(out))
        out = self.softmax(self.fc6(out))
        return out

# Create the model wrapper with three models
model_classes = np.array([Model1(), Model2()])
model_wrapper = network(model_classes)

In [11]:
########### The neural network parametrization ###########
    
dict_net = {
    "GLOBAL": {
        "epochs": 3,
        "loss_function": nn.MSELoss(),
        "optimizer": optim.Adam,
        "scheduler": optim.lr_scheduler.ReduceLROnPlateau,
        "amp": False, ### Mixed precision training
        "quantization": False, ### Quantize the model to e.g. Int8
        "profiler": True,
    },
    "DATA_OPTIONS": {
        "batchsize_schedule": [0],
        "batchsize_training": [None],
        "batchsize_validation": None,
        "shuffle_every_epoch": True,
        "num_workers": 0,
        "pin_memory": None,
        "copy_to_device": False,
    },
    "AMP_OPTIONS" : {
        "dtype": torch.float16,
    },
    "QUANTIZATION_OPTIONS" : {
        "ONNX": {
            "weight_type": QuantType.QInt8,
            "per_channel": False,
        },
        "PYTORCH": {
            "dtype": torch.qint8,
        },
    },
    "LOSS_OPTIONS": {},
    "OPTIMIZER_OPTIONS": {
        "lr": 0.001,
        "weight_decay": 0
        },
    "SCHEDULER_OPTIONS": {
        "patience": 5,
        "factor": 0.5
        },
    "MACHINE_OPTIONS": {
        "device": None,
        "cpu_threads": None,
        "dtype": torch.float32,
    },
    "ONNX": {
        "export_params": True,
        "opset_version": 17,
        "do_constant_folding": True,
        "input_names": ["input"],
        "output_names": ["output"],
        "dynamic_axes": {
            "input": {0: "batch_size"},
            "output": {0: "batch_size"},
        },
    },
    "PROFILER": {
        "schedule": torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=1),
        "on_trace_ready": torch.profiler.tensorboard_trace_handler('./profiler_log'),
        "record_shapes": True,
        "with_stack": True,
    },
    "OTHER": {
        "verbose": True,
        "n_samples": np.infty,
        "save_at_epochs": [1],
    },
}

dict_data = {
    "GLOBAL" : {
        "transform_data" : False,
        "copy_to_device" : True,
    },
    "SCALERS" : {
        "X_data_scalers" : [('yeo-johnson', preprocessing.PowerTransformer(method='yeo-johnson', standardize=True))],
        "Y_data_scalers" : [('yeo-johnson', preprocessing.PowerTransformer(method='yeo-johnson', standardize=True))],
    },
    "MACHINE_OPTIONS" : {
        "device" : None,
        "dtype_X" : torch.float32,
        "dtype_Y" : torch.float32,
        "flexible_data": True,
    },
    "OTHER" : {
        "num_workers" : 0
    }
}


### define the neural network
NeuralNet = NN(network(model_classes))

### data preparation

X_train, X_test, y_train, y_test = train_test_split(input_data,output_data,test_size=0.1,shuffle=True)

data = DataLoading([X_train, y_train], [X_test, y_test], settings = dict_data)

### evaluate training and validation loss over epochs                   
NeuralNet.training(data, settings = dict_net)

NeuralNet.save_net(avoid_q=True)
NeuralNet.save_onnx(example_data=torch.tensor(X_train[:1]))
NeuralNet.eval()

---  Network settings  ---
{
    "GLOBAL": {
        "epochs": 3,
        "loss_function": "MSELoss()",
        "optimizer": "<class 'torch.optim.adam.Adam'>",
        "scheduler": "<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>",
        "amp": false,
        "quantization": false,
        "profiler": true
    },
    "DATA_OPTIONS": {
        "batchsize_schedule": [
            0
        ],
        "batchsize_training": [
            null
        ],
        "batchsize_validation": null,
        "shuffle_every_epoch": true,
        "num_workers": 0,
        "pin_memory": null,
        "copy_to_device": false
    },
    "AMP_OPTIONS": {
        "dtype": "torch.float16"
    },
    "QUANTIZATION_OPTIONS": {
        "ONNX": {
            "weight_type": "QInt8",
            "per_channel": false
        },
        "PYTORCH": {
            "dtype": "torch.qint8"
        }
    },
    "LOSS_OPTIONS": {},
    "OPTIMIZER_OPTIONS": {
        "lr": 0.001,
        "weight_decay": 0
    },
    

AttributeError: 'DataLoader' object has no attribute 'datasetTS'